In [56]:
import pandas as pd

In [57]:
# Importing data
jhu_case_data = pd.read_csv('../csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jhu_death_data = pd.read_csv('../csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

## Analysis for Ft. Carson

In [58]:
# Getting only Colorado
co_case_data = jhu_case_data[jhu_case_data['Province_State']=='Colorado']
co_death_data = jhu_death_data[jhu_death_data['Province_State']=='Colorado']

# Getting rid of stupid FIPS
co_case_data = co_case_data[co_case_data['FIPS']<10000]
co_death_data = co_death_data[co_death_data['FIPS']<10000]

In [59]:
# Looking at the first five lines of the Colorado case data
co_case_data.head(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20
249,84008001.0,US,USA,840,8001.0,Adams,Colorado,US,39.874321,-104.336258,...,1082,1252,1302,1348,1391,1430,1490,1585,1665,1730
250,84008003.0,US,USA,840,8003.0,Alamosa,Colorado,US,37.572506,-105.788545,...,9,10,10,10,10,10,10,10,10,11
251,84008005.0,US,USA,840,8005.0,Arapahoe,Colorado,US,39.649775,-104.335362,...,1824,2030,2083,2160,2252,2312,2389,2472,2557,2625
252,84008007.0,US,USA,840,8007.0,Archuleta,Colorado,US,37.194742,-107.047687,...,8,8,8,8,8,8,8,8,8,8
253,84008009.0,US,USA,840,8009.0,Baca,Colorado,US,37.319409,-102.560322,...,10,10,10,10,10,10,10,10,10,10


In [60]:
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

fips = co_case_data['FIPS']
values = co_case_data[date]

fig = ff.create_choropleth(fips=fips, values=values)
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

fig = ff.create_choropleth(
    fips=fips, values = values, scope=['CO'],
    binning_endpoints = [400,800,1200,1600,2000], 
    colorscale=colorscale,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, round_legend_values=True,
    legend_title=f'Colorado', title= f'Confirmed Cases by County on {date}',
    show_hover=True, centroid_marker={'opacity': 0}
)

fig.layout.template = None
fig.show()

#plt.savefig('test.jpg',format='jpg')

In [61]:
from utils import create_new_key

date_cases = co_case_data[date]
old_date = create_new_key(date, delta)
old_date_cases = co_case_data[old_date]
period_cases = date_cases - old_date_cases

print(f'Total cases in Colorado last {delta} days: {sum(period_cases)}')

Total cases in Colorado last 7 days: 3162


In [62]:
values = period_cases

colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

fig = ff.create_choropleth(
    fips=fips, values = values, scope=['CO'],
    colorscale=colorscale,
    #binning_endpoints = list(range(min(values+1), max(values), round((max(values)-min(values))/4))),
    binning_endpoints = [25,50,100,150,200],
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, round_legend_values=True,
    legend_title=f'Colorado', title= f'Confirmed Cases by County Between {old_date} and {date}',
    show_hover=True, centroid_marker={'opacity': 0}
)

fig.layout.template = None
fig.show()

In [31]:
# Getting data for counties of interest

# List of counties of interest
counties_of_interest = ['Douglas', 'El Paso', 'Pueblo', 'Fremont', 'Teller', 'Elbert', 'Lincoln', 'Crowley', 'Custer']

# Dataframe for only counties of interest
coi_case_data = co_case_data[co_case_data['Admin2'].isin(counties_of_interest)]
coi_death_data = co_death_data[co_death_data['Admin2'].isin(counties_of_interest)]

# Getting death and case data for period
period_coi_cases = coi_case_data[date] - coi_case_data[old_date]
period_coi_deaths = coi_death_data[date] - coi_death_data[old_date]

In [63]:
# Total confirmed cases (date_cases), total confirmed deaths(date_deaths),
# cases in last week (period_cases), deaths in last week (period_deaths)

table_pd = pd.DataFrame({
    'County': coi_case_data['Admin2'],
    f'Total Confirmed Cases {date}': coi_case_data[date],
    f'Last {delta} Days Cases': period_coi_cases,
    f'Total Deaths {date}': coi_death_data[date],
    f'Last {delta} Days Deaths': period_coi_deaths
    })

table_pd

,County,Total Confirmed Cases 5/2/20,Last 7 Days Cases,Total Deaths 5/2/20,Last 7 Days Deaths
262,Crowley,3,1,1,0
263,Custer,2,0,0,0
267,Douglas,518,93,23,4
269,Elbert,31,4,1,0
270,El Paso,988,126,74,2
271,Fremont,22,2,0,0
286,Lincoln,3,-1,0,0
300,Pueblo,150,29,10,1
309,Teller,28,1,2,0


In [64]:
table_pd.to_csv(f"Carsontable-{date.replace('/','-')}-{old_date.replace('/','-')}.csv",index=False)

## Analysis for Ft. Bliss

In [78]:
# Getting Texas + New Mexico
txnm_case_data = jhu_case_data[jhu_case_data['Province_State'].isin(['Texas', 'New Mexico'])]
txnm_death_data = jhu_death_data[jhu_death_data['Province_State'].isin(['Texas', 'New Mexico'])]

# Getting rid of stupid FIPS
txnm_case_data = tx_case_data[tx_case_data['FIPS']<80000]
txnm_death_data = tx_death_data[tx_death_data['FIPS']<80000]

In [81]:
# Looking at the first five lines of the tx/nmcase data
tx_case_data.head(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20
2528,84048001.0,US,USA,840,48001.0,Anderson,Texas,US,31.815347,-95.653548,...,16,18,18,19,23,26,29,29,32,32
2529,84048003.0,US,USA,840,48003.0,Andrews,Texas,US,32.304686,-102.637655,...,19,19,19,19,19,19,19,19,20,20
2530,84048005.0,US,USA,840,48005.0,Angelina,Texas,US,31.254573,-94.609015,...,33,33,36,36,36,46,49,53,54,55
2531,84048007.0,US,USA,840,48007.0,Aransas,Texas,US,28.105562,-96.999505,...,2,2,2,2,2,2,2,2,2,2
2532,84048009.0,US,USA,840,48009.0,Archer,Texas,US,33.615700,-98.687546,...,0,0,0,0,0,0,0,0,0,0


In [80]:
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

fips = tx_case_data['FIPS']
values = tx_case_data[date]

fig = ff.create_choropleth(fips=fips, values=values)
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

fig = ff.create_choropleth(
    fips=fips, values = values, scope=['TX', 'NM'],
    binning_endpoints = [400,800,1200,1600,2000], 
    colorscale=colorscale,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, round_legend_values=True,
    legend_title=f'Texas', title= f'Confirmed Cases by County on {date}',
    show_hover=True, centroid_marker={'opacity': 0}
)

fig.layout.template = None
fig.show()

#plt.savefig('test.jpg',format='jpg')

In [ ]:
from utils import create_new_key

date_cases = co_case_data[date]
old_date = create_new_key(date, delta)
old_date_cases = co_case_data[old_date]
period_cases = date_cases - old_date_cases

print(f'Total cases in Colorado last {delta} days: {sum(period_cases)}')

In [ ]:
values = period_cases

colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

fig = ff.create_choropleth(
    fips=fips, values = values, scope=['CO'],
    colorscale=colorscale,
    #binning_endpoints = list(range(min(values+1), max(values), round((max(values)-min(values))/4))),
    binning_endpoints = [25,50,100,150,200],
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, round_legend_values=True,
    legend_title=f'Colorado', title= f'Confirmed Cases by County Between {old_date} and {date}',
    show_hover=True, centroid_marker={'opacity': 0}
)

fig.layout.template = None
fig.show()

In [ ]:
# Getting data for counties of interest

# List of counties of interest
counties_of_interest = ['El Paso', 'Hudspeth', 'Bell', 'Coryell', 'Lampasas', 'Burnet', 'Williamson', 'Milam', 'Robertson', 'Falls', '' 'Mclennan', 'Bosque']

# Dataframe for only counties of interest
coi_case_data = co_case_data[co_case_data['Admin2'].isin(counties_of_interest)]
coi_death_data = co_death_data[co_death_data['Admin2'].isin(counties_of_interest)]

# Getting death and case data for period
period_coi_cases = coi_case_data[date] - coi_case_data[old_date]
period_coi_deaths = coi_death_data[date] - coi_death_data[old_date]

In [ ]:
# Total confirmed cases (date_cases), total confirmed deaths(date_deaths),
# cases in last week (period_cases), deaths in last week (period_deaths)

table_pd = pd.DataFrame({
    'County': coi_case_data['Admin2'],
    f'Total Confirmed Cases {date}': coi_case_data[date],
    f'Last {delta} Days Cases': period_coi_cases,
    f'Total Deaths {date}': coi_death_data[date],
    f'Last {delta} Days Deaths': period_coi_deaths
    })

table_pd

In [ ]:
table_pd.to_csv(f"Blisstable-{date.replace('/','-')}-{old_date.replace('/','-')}.csv",index=False)

## SANDBOX

In [ ]:
co_data[date].head(5)
co_data[last_week_date].head(5)
last_week_cases.head(5)

In [ ]:
print(list(co_data.keys()))

In [ ]:
# Build figure object (in case fig is a dictionary)
fig = go.Figure(fig)

# Get hover trace
hover_scatter = [scatt for scatt in fig.data if scatt.mode == 'markers'][0]

# Update text to include only county (Not state, FIPS, ec.)
import re
def extract_county(text_val):
    return re.sub(r'County: (.*?)<br>.*', r'\1', text_val)

hover_scatter.text = [extract_county(text_val) for text_val in hover_scatter.text]

# Show text
hover_scatter.mode = 'markers+text'

# Set font properties
hover_scatter.textfont.size = 8

In [42]:
#4/30 - 4/29 for all of CO for 
15284-14758

526

In [143]:
new_key = create_new_key('04/30/20',6)

In [142]:
new_key

'4/24/20'

In [ ]:
import datetime
from datetime import timedelta

def create_new_key(dt, days):
    month, day, year = dt.split('/')
    if len(day) == 1:
        day = '0'+day
    if len(month) ==1:
        month = '0'+month
    old_datetime = datetime.datetime.strptime('/'.join((month,day,year)), '%m/%d/%y')
    new_datetime = old_datetime - timedelta(days=days)
    new_date = new_datetime.strftime('%m/%d/%y')
    month, day, year = new_date.split('/')
    if month[0] == '0':
        month = month[1]
    elif day[0] == '0':
        day = day[1]
    return '/'.join((month, day, year))